# Hierarchical RNN Energy Forecasting Model
This notebook implements a hierarchical RNN model for multi-horizon energy forecasting.

In [3]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import sqlite3
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from typing import Dict, Tuple, List

## 1. Load Data

In [4]:
# Connect to database
conn = sqlite3.connect("energy_data_NE.db")

# Load weather data
weather_query = """
SELECT time as datetime, temperature, dwpt, humidity, precipitation,
       wdir, windspeed, pres, cloudcover
FROM historical_weather_data
"""
weather_data = pd.read_sql_query(weather_query, conn)
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

# Load energy data
energy_data = {
    'solar': pd.read_sql_query("SELECT datetime, value FROM SUN_data_NE", conn),
    'wind': pd.read_sql_query("SELECT datetime, value FROM WND_data_NE", conn),
    'demand': pd.read_sql_query("SELECT datetime, Demand as value FROM demand_data_NE", conn)
}

conn.close()

print("Data loaded successfully!")
print("Weather data shape:", weather_data.shape)
for source, data in energy_data.items():
    print(f"{source} data shape:", data.shape)

Data loaded successfully!
Weather data shape: (17545, 9)
solar data shape: (17545, 2)
wind data shape: (17545, 2)
demand data shape: (17545, 2)


## 2. Define Model Architecture

In [10]:
class TimeSeriesEncoder(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            batch_first=True,
            bidirectional=True
        )

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        # Combine bidirectional states for all batches
        return torch.cat([hidden[-2], hidden[-1]], dim=1)  # [batch, hidden*2]

class HierarchicalEnergyForecaster(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_size: int = 128,
        forecast_horizons: List[int] = [24, 168, 720]  # 24h, 1w, 30d
    ):
        super().__init__()
        self.hidden_size = hidden_size
        self.forecast_horizons = forecast_horizons
        
        # Encoders
        self.short_term_encoder = TimeSeriesEncoder(input_size, hidden_size)
        self.medium_term_encoder = TimeSeriesEncoder(input_size, hidden_size)
        self.long_term_encoder = TimeSeriesEncoder(input_size, hidden_size)
        
        combined_size = hidden_size * 2 * 3
        
        # Prediction heads
        self.prediction_heads = self._create_prediction_heads(combined_size)
        
        # Initialize scalers
        self.scalers = self._initialize_scalers()

    def _create_prediction_heads(self, combined_size):
        return nn.ModuleDict({
            source: nn.ModuleDict({
                str(horizon): self._create_head(combined_size, horizon)
                for horizon in self.forecast_horizons
            })
            for source in ['solar', 'wind', 'demand']
        })

    def _create_head(self, input_size, output_size):
        return nn.Sequential(
            nn.Linear(input_size, self.hidden_size),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(self.hidden_size, output_size)
        )

    def _initialize_scalers(self):
        return {
            'features': MinMaxScaler(),
            'solar': MinMaxScaler(),
            'wind': MinMaxScaler(),
            'demand': MinMaxScaler()
        }
    
    def prepare_data(self, weather_data: pd.DataFrame, energy_data: Dict[str, pd.DataFrame]) -> Tuple:
        """Prepare data for training with multiple time horizons"""
        # Scale features
        features = weather_data[[
            'temperature', 'dwpt', 'humidity', 'precipitation',
            'wdir', 'windspeed', 'pres', 'cloudcover'
        ]]
        
        # Add temporal features if not already present
        if 'hour' not in weather_data.columns:
            weather_data['hour'] = weather_data['datetime'].dt.hour
            weather_data['month'] = weather_data['datetime'].dt.month
            weather_data['season'] = weather_data['datetime'].dt.month.map(
                lambda m: 1 if m in [12, 1, 2] else 2 if m in [3, 4, 5] else 3 if m in [6, 7, 8] else 4
            )
            weather_data['time_of_day'] = weather_data['datetime'].dt.hour.map(
                lambda h: 1 if h < 6 else 2 if h < 12 else 3 if h < 18 else 4
            )
        
        all_features = pd.concat([
            features,
            weather_data[['hour', 'month', 'season', 'time_of_day']]
        ], axis=1)
        
        scaled_features = self.scalers['features'].fit_transform(all_features)
        
        # Create sequences
        sequences = {
            'short_term': self._create_sequences(scaled_features, 24),
            'medium_term': self._create_sequences(scaled_features, 168),
            'long_term': self._create_sequences(scaled_features, 720)
        }
        
        # Prepare targets if energy data is provided
        targets = {}
        if energy_data:
            for source, data in energy_data.items():
                scaled_values = self.scalers[source].fit_transform(data[['value']].values)
                targets[source] = {
                    str(horizon): scaled_values[horizon:] 
                    for horizon in self.forecast_horizons
                }
        
        return sequences, targets

    def _create_sequences(self, data: np.ndarray, sequence_length: int) -> torch.Tensor:
        """Create sequences for a specific time horizon"""
        sequences = []
        for i in range(len(data) - sequence_length):
            sequences.append(data[i:(i + sequence_length)])
        return torch.FloatTensor(np.array(sequences))

    def forward(self, sequences: Dict[str, torch.Tensor]) -> Dict[str, Dict[str, torch.Tensor]]:
        """Forward pass through the model"""
        batch_size = sequences['short_term'].shape[0]
        
        # Process each time scale
        short_term_features = self.short_term_encoder(sequences['short_term'])  # [batch, hidden*2]
        medium_term_features = self.medium_term_encoder(sequences['medium_term'])
        long_term_features = self.long_term_encoder(sequences['long_term'])
        
        # Reshape and combine features
        combined = torch.cat([
            short_term_features.view(batch_size, -1),
            medium_term_features.view(batch_size, -1),
            long_term_features.view(batch_size, -1)
        ], dim=1)  # [batch, hidden*2*3]
        
        # Generate predictions for each source and horizon
        predictions = {}
        for source, heads in self.prediction_heads.items():
            predictions[source] = {}
            for horizon, head in heads.items():
                predictions[source][horizon] = head(combined)
        
        return predictions

print("Model architecture defined!")

Model architecture defined!


## 3. Prepare Data

In [11]:
# Initialize model
input_size = 12  # 8 weather features + 4 temporal features
model = HierarchicalEnergyForecaster(input_size=input_size)

# Prepare sequences and targets
sequences, targets = model.prepare_data(weather_data, energy_data)

print("Data prepared successfully!")
print("\nSequence shapes:")
for horizon, seq in sequences.items():
    print(f"{horizon}:", seq.shape)

: 

In [8]:
def create_batches(sequences, targets, batch_size=32):
    # Calculate minimum length across all sequences
    min_len = min(len(sequences[horizon]) for horizon in sequences)
    
    # Number of complete batches
    n_batches = min_len // batch_size
    
    for i in range(n_batches):
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        # Prepare batch sequences
        batch_sequences = {
            horizon: sequences[horizon][start_idx:end_idx]
            for horizon in sequences
        }
        
        # Prepare batch targets
        batch_targets = {
            source: {
                str(horizon): targets[source][str(horizon)][start_idx:end_idx]
                for horizon in model.forecast_horizons
            }
            for source in ['solar', 'wind', 'demand']
        }
        
        yield batch_sequences, batch_targets

## 4. Train Model

In [9]:
# Modified training loop
batch_size = 32
epochs = 50
learning_rate = 0.001

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

losses = []
for epoch in range(epochs):
    epoch_loss = 0
    batch_count = 0
    
    for batch_sequences, batch_targets in create_batches(sequences, targets, batch_size):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(batch_sequences)
        
        # Calculate loss
        loss = sum(
            criterion(
                predictions[source][str(horizon)],
                torch.FloatTensor(batch_targets[source][str(horizon)])
            )
            for source in ['solar', 'wind', 'demand']
            for horizon in model.forecast_horizons
        )
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        batch_count += 1
    
    avg_epoch_loss = epoch_loss / batch_count
    losses.append(avg_epoch_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Average Loss: {avg_epoch_loss:.4f}')

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(losses)
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (192x128 and 768x128)

## 5. Save Model

In [ ]:
# Save model
torch.save(model.state_dict(), 'hierarchical_rnn_model.pkl')
print("Model saved successfully!")

## 6. Test Predictions

In [ ]:
# Make predictions for a sample period
test_data = weather_data.iloc[-48:].copy()  # Last 48 hours
predictions = model.predict(test_data)

# Plot predictions
plt.figure(figsize=(15, 5))
plt.plot(predictions['solar']['24'], label='Solar (24h)')
plt.plot(predictions['wind']['24'], label='Wind (24h)')
plt.plot(predictions['demand']['24'], label='Demand (24h)')
plt.title('24-hour Predictions')
plt.legend()
plt.show()